# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe82416feb0>
├── 'a' --> tensor([[ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158]])
└── 'x' --> <FastTreeValue 0x7fe82416f9a0>
    └── 'c' --> tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                        [-1.9460,  0.0126, -0.7483,  0.6078],
                        [-0.4267,  0.9315,  0.4827,  0.8824]])

In [4]:
t.a

tensor([[ 0.6878, -0.0334,  1.2293],
        [ 0.6935, -0.6508,  0.0158]])

In [5]:
%timeit t.a

74.6 ns ± 0.834 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe82416feb0>
├── 'a' --> tensor([[-1.0723, -0.1093, -3.5996],
│                   [ 0.2824, -1.4736,  0.4451]])
└── 'x' --> <FastTreeValue 0x7fe82416f9a0>
    └── 'c' --> tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                        [-1.9460,  0.0126, -0.7483,  0.6078],
                        [-0.4267,  0.9315,  0.4827,  0.8824]])

In [7]:
%timeit t.a = new_value

77.8 ns ± 0.412 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6878, -0.0334,  1.2293],
               [ 0.6935, -0.6508,  0.0158]]),
    x: Batch(
           c: tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6878, -0.0334,  1.2293],
        [ 0.6935, -0.6508,  0.0158]])

In [11]:
%timeit b.a

68.5 ns ± 0.893 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2017,  0.0075, -0.2302],
               [ 1.0406,  0.1865,  0.0732]]),
    x: Batch(
           c: tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824]]),
       ),
)

In [13]:
%timeit b.a = new_value

613 ns ± 6.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

954 ns ± 12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 50.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

206 µs ± 9.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 949 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe8200e7ac0>
├── 'a' --> tensor([[[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]],
│           
│                   [[ 0.6878, -0.0334,  1.2293],
│                    [ 0.6935, -0.6508,  0.0158]]])
└── 'x' --> <FastTreeValue 0x7fe774dc31f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe774de5f40>
├── 'a' --> tensor([[ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158],
│                   [ 0.6878, -0.0334,  1.2293],
│                   [ 0.6935, -0.6508,  0.0158]])
└── 'x' --> <FastTreeValue 0x7fe774de6f40>
    └── 'c' --> tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                        [-1.9460,  0.0126, -0.7483,  0.6078],
                 

In [23]:
%timeit t_cat(trees)

36 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2535,  0.8150,  0.2030,  0.2098],
                       [-1.9460,  0.0126, -0.7483,  0.6078],
                       [-0.4267,  0.9315,  0.4827,  0.8824]],
              
                      [[-0.2535,  0.8150,  0.2030,  0.2098],
                       [-1.9460,  0.0126, -0.7483,  0.6078],
                       [-0.4267,  0.9315,  0.4827,  0.8824]],
              
                      [[-0.2535,  0.8150,  0.2030,  0.2098],
                       [-1.9460,  0.0126, -0.7483,  0.6078],
                       [-0.4267,  0.9315,  0.4827,  0.8824]],
              
                      [[-0.2535,  0.8150,  0.2030,  0.2098],
                       [-1.9460,  0.0126, -0.7483,  0.6078],
                       [-0.4267,  0.9315,  0.4827,  0.8824]],
              
                      [[-0.2535,  0.8150,  0.2030,  0.2098],
                       [-1.9460,  0.0126, -0.7483,  0.6078],
                       [-0.4267,  0.9315,  0.4827,  0.8824]],

In [26]:
%timeit Batch.stack(batches)

330 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824],
                      [-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824],
                      [-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824],
                      [-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824],
                      [-0.2535,  0.8150,  0.2030,  0.2098],
                      [-1.9460,  0.0126, -0.7483,  0.6078],
                      [-0.4267,  0.9315,  0.4827,  0.8824],
                      [-0.2535,  0.8150,  0.2030,  0.2098],
                   

In [28]:
%timeit Batch.cat(batches)

165 µs ± 818 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

381 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
